In [204]:
# Pima Indians Diabetes Database Analysis
#The datasets consist of several medical predictor (independent) variables and one target (dependent) variable, Outcome. Independent variables include the number of pregnancies the patient has had, their BMI, insulin level, age, and so on
#Source - https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import torch.nn.functional as F
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms
import seaborn as sns

dataset = 'ouPima_Diabetes/diabetes.csv'
df = pd.read_csv(dataset)
dfh = df.head()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(' Using {} device'.format(device))

 Using cpu device


In [205]:
dfh

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [206]:
df.shape

(768, 9)

In [207]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [208]:
train_data = df.iloc[:, 0:8]
test_data = df.iloc[:,[8]]

In [209]:
# #Define and setup the dataset using pytorch inbuilt functions
# train_data = torch.from_numpy(np.asarray(train_data))
# test_data = torch.from_numpy(np.asarray(test_data))
# train_ds = TensorDataset(train_data,test_data)

#Define and setup the dataset using pytorch inbuilt functions
train_data = torch.tensor(np.asarray(train_data),dtype=torch.float32)
test_data = torch.tensor(np.asarray(test_data),dtype=torch.float32)
combined_ds = TensorDataset(train_data,test_data)

In [210]:
print(train_data.dtype)
print(test_data.dtype)

torch.float32
torch.float32


In [211]:
combined_ds[0:3]

(tensor([[  6.0000, 148.0000,  72.0000,  35.0000,   0.0000,  33.6000,   0.6270,
           50.0000],
         [  1.0000,  85.0000,  66.0000,  29.0000,   0.0000,  26.6000,   0.3510,
           31.0000],
         [  8.0000, 183.0000,  64.0000,   0.0000,   0.0000,  23.3000,   0.6720,
           32.0000]]),
 tensor([[1.],
         [0.],
         [1.]]))

In [212]:
#Define and setup the DataLoader

batch_size = 5
train_ds = DataLoader(train_data, batch_size, shuffle=True)
test_ds = DataLoader(test_data, batch_size, shuffle=True)
combined_dl = DataLoader(combined_ds, batch_size, shuffle=True)
for xi,yi in combined_dl:
    print(xi)
    print(yi)
    break

tensor([[  1.0000, 100.0000,  66.0000,  29.0000, 196.0000,  32.0000,   0.4440,
          42.0000],
        [  3.0000,  61.0000,  82.0000,  28.0000,   0.0000,  34.4000,   0.2430,
          46.0000],
        [  1.0000, 124.0000,  60.0000,  32.0000,   0.0000,  35.8000,   0.5140,
          21.0000],
        [  5.0000, 111.0000,  72.0000,  28.0000,   0.0000,  23.9000,   0.4070,
          27.0000],
        [  1.0000,  96.0000,  64.0000,  27.0000,  87.0000,  33.2000,   0.2890,
          21.0000]])
tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.]])


In [213]:
#Define the model for weights and bias using in-built nn.Linear which does ot automatically

class DiabetesDetectionNet(nn.Module):
    
    def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):
        
        super(DiabetesDetectionNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)
        #self.relu3 = nn.ReLU()
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        #out = self.relu3(out)
        return out



In [214]:
model = DiabetesDetectionNet(8, 100, 50, 2)
print(model)

DiabetesDetectionNet(
  (fc1): Linear(in_features=8, out_features=100, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)


In [216]:
#Define Loss Function

import torch.nn.functional as F

loss_fn = nn.CrossEntropyLoss()

#loss = loss_fn(model(train_data), test_data)
#print(loss)



In [217]:
#Define Optimizer/Algorithm - SGD (GD) & Train the model

opt1 = torch.optim.SGD(model.parameters(), lr=0.001)

opt2 = torch.optim.Adam(model.parameters(), lr=0.001)


def Fit(num_epochs, model, loss_fn, opt, combined_dl):
    
    for epoch in range(num_epochs):
        
        for xb,yb in combined_dl:
            
            pred = model(xb)
            loss = loss_fn(pred, yb)
            loss.backward()
            opt.step()
            opt.zero_grad()
        
        if (epoch+1) % 10 ==0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))




In [218]:
Fit(500, model, loss_fn, opt2, combined_dl)

RuntimeError: 0D or 1D target tensor expected, multi-target not supported

In [191]:
print("targets are",test_data)
print("predictions are", model(train_data))

#Since test_data is binary and train is not - we likely need to add activation function that converts the output to binary or so

targets are tensor([[1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        

In [143]:
model(train_data)

tensor([[0.0000],
        [0.0000],
        [0.0000],
        [0.1075],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.7393],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.4179],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.4712],
        [0.0000],
        [0.0000],
        [0.0000],
        [0.0000],
        [0